In [7]:
# LightGBM 라이브러리 설치
# !pip3 install lightgbm

In [8]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# -------------------------------------------------------
# 1. 핵심 변수 골라내기 (SHAP > 0)
# -------------------------------------------------------
# SHAP 분석 결과 파일 불러오기
# (파일 이름은 네가 가지고 있는 걸로 맞춰줘!)
feature_file = "../VIP_Target/Target_data_VIP_shap_vif_ascending.csv" 
shap_df = pd.read_csv(feature_file)

# SHAP 중요도가 0보다 큰 변수 이름만 리스트로 뽑기
selected_features = shap_df[shap_df['SHAP_Importance'] > 0]['Feature'].tolist()

print(f"✅ 전체 변수 중 SHAP 0 제외하고 '{len(selected_features)}개'의 핵심 변수만 사용합니다.")

# -------------------------------------------------------
# 2. 전체 데이터 불러오기 & 학습 데이터 준비
# -------------------------------------------------------
# 전체 데이터 (Target 포함된 파일)
data_file = "../VIP_Target/VIP_With_Target.csv"
df = pd.read_csv(data_file)

# 선택된 변수(X)와 정답(y) 나누기
X = df[selected_features]
y = df['Target']

# 빈칸(NaN)은 0으로 채우기 (LightGBM은 결측치 처리도 잘하지만 안전하게!)
X = X.fillna(0)

# 훈련용(Train)과 테스트용(Test)으로 나누기 (8:2 비율)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"학습 데이터: {X_train.shape}, 테스트 데이터: {X_test.shape}")

# -------------------------------------------------------
# 3. LightGBM 모델 학습
# -------------------------------------------------------
print("\n🚀 LightGBM 모델 학습 시작...")

model = lgb.LGBMClassifier(
    n_estimators=200,      # 반복 횟수
    learning_rate=0.05,    # 학습 속도
    max_depth=10,          # 나무 깊이 (너무 깊으면 과적합되니 적당히)
    random_state=42,
    n_jobs=-1              # 모든 CPU 코어 사용 (속도 UP)
)

model.fit(X_train, y_train)
print("학습 완료!")

# -------------------------------------------------------
# 4. 성능 평가 (성적표 확인)
# -------------------------------------------------------
y_pred = model.predict(X_test)

print("\n" + "="*40)
print("[ LightGBM 모델 성능 평가 결과 ]")
print("="*40)
print(f"정확도 (Accuracy): {accuracy_score(y_test, y_pred):.4f}")
print("-" * 40)
print("상세 보고서:\n")
print(classification_report(y_test, y_pred))
print("-" * 40)
print("혼동 행렬 (Confusion Matrix):\n")
print(confusion_matrix(y_test, y_pred))

✅ 전체 변수 중 SHAP 0 제외하고 '103개'의 핵심 변수만 사용합니다.
학습 데이터: (1528, 103), 테스트 데이터: (382, 103)

🚀 LightGBM 모델 학습 시작...
[LightGBM] [Info] Number of positive: 180, number of negative: 1348
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003719 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20067
[LightGBM] [Info] Number of data points in the train set: 1528, number of used features: 103
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.117801 -> initscore=-2.013420
[LightGBM] [Info] Start training from score -2.013420
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

In [9]:
df.head()

,기준년월,발급회원번호,증감율_이용건수_신용_전월,증감율_이용건수_신판_전월,증감율_이용건수_일시불_전월,증감율_이용건수_할부_전월,증감율_이용건수_CA_전월,증감율_이용건수_체크_전월,증감율_이용건수_카드론_전월,증감율_이용금액_신용_전월,...,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월,Slope_Spend,Slope_Balance,Slope_Count,Churn_Score,Cluster_ID,Target
0,201812,SYN_0,-0.048162,-0.048162,-0.048162,0.0,0.0,0.000000,0.0,0.238954,...,0,0,5.자녀성장기(2),27,-11116.685714,7214.657143,-0.571429,4.2,2,0
1,201812,SYN_1000,0.016580,0.016580,0.016580,0.0,0.0,0.000000,0.0,0.279102,...,0,0,4.자녀성장기(1),50,-15564.771429,-17170.742857,-2.428571,13.5,1,0
2,201812,SYN_1005,-0.094536,-0.094536,-0.094536,0.0,0.0,-0.037096,0.0,0.214526,...,0,0,2.가족형성기,13,-65200.657143,-10257.428571,-0.057143,11.4,1,0
3,201812,SYN_1007,-0.008564,-0.008564,-0.008564,0.0,0.0,0.000000,0.0,0.221973,...,0,0,3.자녀출산기,5,-3197.514286,-52205.085714,-1.000000,7.2,1,0
4,201812,SYN_1009,0.017918,0.017918,0.017918,0.0,0.0,-0.132413,0.0,0.239055,...,0,0,5.자녀성장기(2),27,4787.828571,-44019.000000,0.657143,2.1,2,0
